In [ ]:
%%HTML
<script src = "require.js"></script>

In [ ]:

import sys
sys.path.append('../data')
sys.path.append('../Librerias')

import dataset as ds
import utils
import caract as dc


import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import seaborn as sns
import copy

from os import listdir
from os.path import isfile, join

import scipy.signal as signal
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"

sns.set_style("darkgrid")

In [ ]:
#fpath = "/Users/granosftp/Documents/GitHub/Tesis/data/high/"
fpath =  "/Users/consu/OneDrive/Documentos/GitHub/Tesis/data/low/"
onlyfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]

cutoff = [8/1000, 11/1000]


## **frames**

In [ ]:
#load data
frames = []
for fname in onlyfiles:
    if fname.endswith(".mat"):
        set =  ds.MatFileToDataFrame(fpath, fname)
        df = set.get_dataframe(cutoff)
        frames.append(df)
        print(fname)

In [ ]:
defectos_frames = []
taus = []
for i, df in enumerate(frames):
    aux = dc.CaractDefect(df)
    defectos_frames.append(aux)
    taus.append(aux.get_tau()[1])
    sys.stdout.write(str(i) + '\r')

In [ ]:
hilbert_frames = []
for i, df in enumerate(defectos_frames):
    aux, _ = df.get_hilbert()
    hilbert_frames.append(aux)
    sys.stdout.write(str(i) + '\r')

In [ ]:
# all frames
frames_cd = []

for i in range(len(frames)):
    frames_cd.append(pd.merge(frames[i], hilbert_frames[i], on='Hilbert Transform', how='outer'))

frames_cd[0]

## **extracción de amplitudes**

In [ ]:
# test con 1 archivo
x = frames_cd[0]

max = x['Amplitude'].max()
num = [2, 10, 15, 20, 25, 30, 35, 40, 45, 50]


In [ ]:
max*2/100

In [ ]:
all_indx = []
for elem in num:
    aux = []
    for i, amp in enumerate(x['Amplitude']):
        if amp <= max*elem/100:
            aux.append(i)
    all_indx.append(aux)

In [ ]:
len(all_indx[-1])

In [ ]:
amplitudes = pd.DataFrame()

for i, indx in enumerate(all_indx):
    aux = x.drop(indx)
    amplitudes[f'amp_{num[i]}'] = aux['Amplitude']
    

In [ ]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=x['Amplitude'].index, y=x['Amplitude'], name='Amplitud OG'))
#fig.add_trace(go.Scatter(x = x['Amplitude'].index, y = Sxx*f**(5), name = 'f^(5/2)'))
for i, elem in enumerate(amplitudes.columns):
    fig.add_trace(go.Scatter(x=x['Amplitude'].index, y=amplitudes[elem], name=f'amp_{num[i]}'))

fig.update_layout(
    title = 'PSD',
    xaxis_title='Frequency [Hz]',
    yaxis_title='Spectrum [V RMS]',
    legend_title='Legend'
)

fig.show()

In [ ]:
f, Sxx = signal.welch(x['Amplitude'], fs = 1000, nperseg = 2000, scaling='spectrum', window='hamming')

In [ ]:
welch = pd.DataFrame()
#welch['freq'] = f
#welch['OG'] = Sxx
freqs = []
for i, elem in enumerate(amplitudes.columns):
    xx = amplitudes[elem].dropna()
    f, welch[f'welch_{num[i]}'] = signal.welch(xx, fs =1000, nperseg = 2000, scaling = 'spectrum', window='hamming')
    freqs.append(f)



In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=f, y=Sxx*f**5, name='Amplitud OG'))
fig.add_trace(go.Scatter(x = f, y = Sxx*f**(5), name = 'f^(5)'))
for i, elem in enumerate(welch.columns):
    fig.add_trace(go.Scatter(x=freqs[i], y=welch[elem]*f**5, name=f'amp_{num[i]}'))


fig.update_layout(
    title = 'PSD señal por Welch archivo 0',
    xaxis_title='Frequency [Hz]',
    yaxis_title='Spectrum [V RMS]',
    legend_title='Legend'
)

fig.update_xaxes(type = 'log')
fig.update_yaxes(type = 'log')

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=f, y=Sxx*f**4, name='Amplitud OG'))
fig.add_trace(go.Scatter(x = f, y = Sxx*f**(4), name = 'f^(4)'))
for i, elem in enumerate(welch.columns):
    fig.add_trace(go.Scatter(x=freqs[i], y=welch[elem]*f**4, name=f'amp_{num[i]}'))


fig.update_layout(
    title = 'PSD señal por Welch archivo 0',
    xaxis_title='Frequency [Hz]',
    yaxis_title='Spectrum [V RMS]',
    legend_title='Legend'
)

fig.update_xaxes(type = 'log')
fig.update_yaxes(type = 'log')

fig.show()

In [ ]:
#welch.to_csv('welch_amplitudes.csv')